In [1]:
import pandas as pd
import numpy as np
import random
random.seed(42)
from datetime import datetime, timedelta
from sklearn import preprocessing
%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import evaluate_with_xgboost
from mpmf.utils import get_top_1_motif_numba

In [2]:
def New_preprocessing(
    TimeSeries,
    num_periods_input,
    num_periods_output,
    include_itself,
    include_covariates,
    include_motif_information,
    k_motifs,
    no_points_after_motif,
    do_normalization=False,
    include_similarity=False
):
    Data = []
    # Change 1
    #################################################################################################
    start_date = datetime(2012, 5, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])  # adding the pemds7 value
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.minute)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])
        start_date = start_date + timedelta(minutes=5)
        Data.append(record)
    headers = [
        "pems",
        "month",
        "day",
        "hour",
        "minute",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
    Data_df = pd.DataFrame(Data, columns=headers)
    # print("Shape of Data_df:", Data_df.shape)
    sub = Data_df.iloc[:, 1:]
    # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
    New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
    # print("Shape of New_sub:", New_sub.shape)
    Normalized_Data_df = pd.DataFrame(
        np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
    )
    #################################################################################################
    if include_motif_information:
        if include_motif_information == 1 or include_motif_information == 2:  # get_top_1_motif
            df_motif = get_top_1_motif_numba(
                TimeSeries,
                num_periods_output,
                l=no_points_after_motif,
                include_itself=include_itself,
                compute_trend=(include_motif_information == 2)
            )

            df_motif_points = df_motif[[c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))]]

            if do_normalization:
                print("Applying z-score normalization to motif points...:", df_motif_points)
                df_motif_points = df_motif_points.sub(df_motif_points.mean(axis=1), axis=0).div(df_motif_points.std(axis=1, ddof=0), axis=0) # Use Population Standard Deviation (ddof=0)
                df_motif_points = df_motif_points.replace([np.inf, -np.inf], np.nan)
                print("After replacing inf values with NaN:", df_motif_points)
            
            Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif_points], axis=1)

        if include_similarity:
            df_motif_dist = df_motif[[c for c in df_motif.columns if ("dist" in c)]]
            # print("Shape of df_motif_dist before normalization:", df_motif_dist.shape)
            df_motif_dist = pd.DataFrame(preprocessing.minmax_scale(df_motif_dist, feature_range=(-0.5, 0.5)), columns=df_motif_dist.columns, index=df_motif_dist.index)
            # print("Shape of df_motif_dist after normalization:", df_motif_dist.shape)
            Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif_dist], axis=1)

        # if include_motif_information == 2:  # get_top_k_motifs (Direct)
        #     df_motif = get_top_k_motifs(
        #         TimeSeries,
        #         num_periods_output,
        #         k=k_motifs,
        #         l=no_points_after_motif,
        #         include_itself=include_itself,
        #     )
        #     interested_features = [
        #         c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
        #     ]
        #     df_motif = df_motif[interested_features]
        # if include_motif_information == 3:  # get_top_k_motifs (Unweighted Average)
        #     df_motif = get_top_k_motifs(
        #         TimeSeries,
        #         num_periods_output,
        #         k=k_motifs,
        #         l=no_points_after_motif,
        #         include_itself=include_itself,
        #     )
        #     interested_features = [c for c in df_motif.columns if ("idx" not in c)]
        #     df_motif = df_motif[interested_features]
        #     df_motif = compute_point_after_average(df_motif)
        # if include_motif_information == 4:  # get_top_k_motifs (Weighted Average)
        #     df_motif = get_top_k_motifs(
        #         TimeSeries,
        #         num_periods_output,
        #         k=k_motifs,
        #         l=no_points_after_motif,
        #         include_itself=include_itself,
        #     )
        #     interested_features = [c for c in df_motif.columns if ("idx" not in c)]
        #     df_motif = df_motif[interested_features]
        #     df_motif = compute_point_after_average(df_motif, method="weighted")

    #################################################################################################
    # Change 2
    split_index = 11232
    #################################################################################################
    Train = Normalized_Data_df.iloc[0:split_index, :]
    Train = Train.values
    Train = Train.astype("float32")
    Test = Normalized_Data_df.iloc[(split_index - num_periods_input) :, :]
    Test = Test.values
    Test = Test.astype("float32")
    Number_Of_Features = Normalized_Data_df.shape[1]
    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    while next + (num_periods_input + num_periods_output) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)
    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input + num_periods_output) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + 1
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)

    if include_covariates and include_motif_information:
        pass
    elif include_covariates and (not include_motif_information):
        pass
    elif (not include_covariates) and include_motif_information:
        selected_cols = np.r_[
            0, len(headers) : Normalized_Data_df.shape[1]
        ]
        x_batches = x_batches[:, :, selected_cols]
        x_testbatches = x_testbatches[:, :, selected_cols]
    else: # (not include_covariates) and (not include_motif_information)
        pass
    return x_batches, y_batches, x_testbatches, y_testbatches

In [3]:
# --- Warmup Run ---
# print("Performing Numba warmup (ignoring time)...")
# # This compiles the function so the next run is fast
# _ = get_top_1_motif_numba(np.random.rand(100), m=10)
# print("Warmup complete.\n")

file_name = "pems.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
data = pd.DataFrame(data)
# data = data[:5]
print("Data shape:", data.shape)

num_periods_input = 9  # input
num_periods_output = 9  # to predict


# ================== Parameters that are going to be changed==================
include_covariates = True  # True/False : whether to include features or not
include_motif_information = 1  # 0: no motif info; 1: Top-1 Motif; 2: Top-K Motifs (Direct); 3: Top-K Motifs (Average); 4: Top-K Motifs (Weighted Average)
no_points_after_motif = 9  # number of points to consider: 1, ceil(m/2), m
do_normalization = True
include_similarity = True
# ================================================
include_itself = True
k_motifs = 1  # 1, 3, 5
# ================================================

xgboost_parameters = {
    "learning_rate": 0.045,
    "n_estimators": 150,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    "random_state": 42,
    "verbosity": 1,  # 0=Silent, 1=Warning, 2=Info, 3=Debug
}

# ALL_Test_Data = []
# ALL_Test_Prediction = []

Data shape: (228, 12672)


In [ ]:
# =================== Processing Time Series ===================
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
for i in range(0, len(data)):
    print("Processing Time Series:", i)
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(
        TimeSeries,
        num_periods_input,
        num_periods_output,
        include_itself,
        include_covariates,
        include_motif_information,
        k_motifs,
        no_points_after_motif,
        do_normalization,
        include_similarity
    )
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)
## =================== End of Processing Time Series ===================
# Error 136: ValueError: could not broadcast input array from shape (8,) into shape (9,)

Processing Time Series: 0


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Applying z-score normalization to motif points...:        top_1_motif_point_after_1  top_1_motif_point_after_2  \
0                            NaN                        NaN   
1                            NaN                        NaN   
2                            NaN                        NaN   
3                            NaN                        NaN   
4                            NaN                        NaN   
...                          ...                        ...   
12667                       68.9                       68.8   
12668                       18.0                       16.8   
12669                       16.8                       16.9   
12670                       16.9                       18.4   
12671                       22.6                       22.0   

       top_1_motif_point_after_3  top_1_motif_point_after_4  \
0                            NaN                        NaN   
1                            NaN                        NaN   
2  

In [ ]:
rmse, wape, mae, mape = evaluate_with_xgboost(
    num_periods_output,
    x_batches_Full,
    y_batches_Full,
    X_Test_Full,
    Y_Test_Full,
    xgboost_parameters,
    (include_covariates or (include_motif_information > 0)),
)
print("RMSE: ", rmse)
print("WAPE: ", wape)
print("MAE: ", mae)
print("MAPE: ", mape)

RMSE:  5.1625695
WAPE:  0.04769469
MAE:  2.7791035
MAPE:  0.067805


In [ ]:
import datetime

print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")

This notebook was last run end-to-end on: 2026-01-11 02:28:30.291593

